In [ ]:
#using spacy as backend
#https://pypi.org/project/rasa-nlu/
!pip install rasa_nlu[spacy]

In [ ]:
import sys
!{sys.executable} -m spacy download en_core_web_md

In [ ]:
!{sys.executable} -m spacy link en_core_web_md en

In [4]:
# Load the Packages
from rasa_nlu.training_data  import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

In [5]:
# view data
!cat rasa_dataset.json

{
  "rasa_nlu_data": {
    "regex_features": [
      {
        "name": "zipcode",
        "pattern": "[0-9]{5}"
      },
      {
        "name": "greet",
        "pattern": "hey[^\\s]*"
      }
    ],
    "entity_synonyms": [
      {
        "value": "chinese",
        "synonyms": ["Chinese", "Chines", "chines"]
      },
      {
        "value": "vegetarian",
        "synonyms": ["veggie", "vegg"]
      }
    ],
    "common_examples": [
      {
        "text": "hey", 
        "intent": "greet", 
        "entities": []
      }, 
      {
        "text": "howdy", 
        "intent": "greet", 
        "entities": []
      }, 
      {
        "text": "hey there",
        "intent": "greet", 
        "entities": []
      }, 
      {
        "text": "hello", 
        "intent": "greet", 
        "entities": []
      }, 
      {
        "text": "hi", 
        "intent": "greet", 
        "entities": []
      },
      {
        "text": "good morning",
        "intent": "greet",
        "entities": 

In [6]:
#load the dataset
train_data = load_data('rasa_dataset.json')

In [7]:
# Config Backend using Sklearn and Spacy
trainer = Trainer(config.load("config_spacy.yaml"))

In [8]:
#training data
trainer.train(train_data)

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.1s finished


In [9]:
# Returns the directory the model is stored in (inside projects)
model_directory = trainer.persist('projects/')

### Before going to the model prediction with rasa nlu, lets see how it works in Spacy

In [10]:
import spacy
nlp = spacy.load('en')

In [11]:
document = nlp(u"I am looking for a Finnish restaurant in the center of town")

In [12]:
for word in document.ents:
    print("value",word.text,"entity",word.label_,"start",word.start_char,"end",word.end_char)

value Finnish entity NORP start 19 end 26


In [13]:
spacy.explain('NORP')

'Nationalities or religious or political groups'

### Making prediction with model

In [14]:
from rasa_nlu.model import Metadata, Interpreter

In [15]:
#loading the model
# model_directory points to the folder the model is present
interpreter = Interpreter.load(model_directory)

In [16]:
#Prediction of the intent
interpreter.parse(u"I am looking for a Finnish restaurant in the center of town")

{'intent': {'name': 'restaurant_search', 'confidence': 0.7470537828630842},
 'entities': [],
 'intent_ranking': [{'name': 'restaurant_search',
   'confidence': 0.7470537828630842},
  {'name': 'goodbye', 'confidence': 0.15190494620620407},
  {'name': 'affirm', 'confidence': 0.051965765552677394},
  {'name': 'greet', 'confidence': 0.049075505378034415}],
 'text': 'I am looking for a Finnish restaurant in the center of town'}

In [17]:
interpreter.parse(u"I want an Nepalese Spot to eat")

{'intent': {'name': 'restaurant_search', 'confidence': 0.6984222685518969},
 'entities': [],
 'intent_ranking': [{'name': 'restaurant_search',
   'confidence': 0.6984222685518969},
  {'name': 'goodbye', 'confidence': 0.18938850298467455},
  {'name': 'greet', 'confidence': 0.0581335996817117},
  {'name': 'affirm', 'confidence': 0.05405562878171704}],
 'text': 'I want an Nepalese Spot to eat'}

In [18]:
interpreter.parse(u"Hello Awesome People")

{'intent': {'name': 'affirm', 'confidence': 0.44722502119386587},
 'entities': [],
 'intent_ranking': [{'name': 'affirm', 'confidence': 0.44722502119386587},
  {'name': 'greet', 'confidence': 0.2541951211318638},
  {'name': 'goodbye', 'confidence': 0.15500171775201005},
  {'name': 'restaurant_search', 'confidence': 0.1435781399222602}],
 'text': 'Hello Awesome People'}

#### We can see that the model predicts the intent according to the input given to it